In [38]:
import operator
import math
import random
import sympy
import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

In [43]:

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(math.cos, 1)
pset.addPrimitive(math.sin, 1)
pset.addEphemeralConstant("rand", lambda: random.randint(-1,1))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = ((func(x) - x**4 - x**3 - x**2 - x)**2 for x in points)
    return math.fsum(sqerrors) / len(points),

toolbox.register("evaluate", evalSymbReg, points=[x/10. for x in range(-10,10)])
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)
    
    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()

D:\SoftwareAPPs\Anaconda\AnacondaInstaller\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
D:\SoftwareAPPs\Anaconda\AnacondaInstaller\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


   	      	                        fitness                        	                      size                     
   	      	-------------------------------------------------------	-----------------------------------------------
gen	nevals	avg    	gen	max  	min     	nevals	std    	avg    	gen	max	min	nevals	std    
0  	300   	1.78879	0  	30.34	0.450825	300   	2.67896	3.54667	0  	7  	2  	300   	1.49482
1  	166   	1.43254	1  	44.4437	0.183711	166   	3.05668	3.60667	1  	12 	1  	166   	1.77725
2  	166   	2.16879	2  	315.736	0.165572	166   	18.1873	3.55   	2  	9  	1  	166   	1.62506
3  	163   	0.98255	3  	2.9829 	0.165572	163   	0.712666	3.42667	3  	9  	1  	163   	1.45073
4  	153   	0.836017	4  	14.538 	0.165572	153   	0.979399	3.77   	4  	11 	1  	153   	1.64025
5  	158   	0.944635	5  	18.9739	0.165572	158   	1.61614 	3.77667	5  	10 	1  	158   	1.62894
6  	169   	0.885819	6  	14.2181	0.165572	169   	1.00296 	4      	6  	10 	1  	169   	1.87617
7  	167   	0.731332	7  	3.35292	0.165572	167   

In [90]:
hof_tree = str(gp.PrimitiveTree(hof)[0])
hof_tree_ex = sympy.sympify(hof_tree, locals=converter)
hof_tree_ex

x*(x*(x**2 + x) + x) + x

In [89]:
sympy.simplify(hof_tree_ex)

x*(x*(x*(x + 1) + 1) + 1)

In [91]:
sympy.expand(hof_tree_ex)

x**4 + x**3 + x**2 + x

In [65]:
individual_1 = pop[161]
tree_1 = gp.PrimitiveTree(individual_1)
str(tree_1)
sympy.sympify(str(tree), locals=converter)

cos(sin(x**2*cos(x) + x - cos(x**2) - 1 - cos(1) + sin(1))*sin(cos(sin(x))))

In [32]:
expr = gp.genHalfAndHalf(pset, min_=4, max_=10)
expr

In [33]:
tree = gp.PrimitiveTree(expr)

In [17]:
tree

[<deap.gp.Primitive at 0x21943166270>, <deap.gp.Terminal at 0x21941565f00>]

In [34]:
str(tree)

'cos(mul(sin(neg(sub(sub(sub(add(sub(x, x), x), 1), sub(sub(cos(1), sin(1)), mul(cos(x), mul(x, x)))), cos(mul(x, x))))), sin(neg(neg(cos(sin(x)))))))'

In [35]:
function = gp.compile(tree, pset)


In [36]:
function(1)

0.9832363969565644

In [79]:
converter = {
            'sub': lambda x, y: x - y,
            'div': lambda x, y: x / y,
            'mul': lambda x, y: x * y,
            'add': lambda x, y: x + y,
            'neg': lambda x: -x,
            'abs': lambda x: sympy.Abs(x),
            'pow': lambda x, y: x**y,
            'sin': lambda x: sympy.sin(x),
            'cos': lambda x: sympy.cos(x),
            'inv': lambda x: 1 / x,
            'sqrt': lambda x: x**0.5,
            'self_exp': lambda x: sympy.exp(x**2),
            'square': lambda x: x**2
        }  # Converter set

        
# next_e = sympy.sympify(str(tree), locals=converter)
next_e = sympy.sympify('mul(x, x)', locals=converter)
print('Expression:', next_e)
next_e

Expression: x**2


x**2

In [74]:
hof_tree = str(gp.PrimitiveTree(hof)[0])
sympy.simplify(hof_tree, locals=converter)

add(mul(x, sub(x, neg(mul(x, sub(x, neg(mul(x, x))))))), x)

In [86]:
converter = {
            'sub': lambda x, y: x - y,
            'div': lambda x, y: x / y,
            'mul': lambda x, y: x * y,
            'add': lambda x, y: x + y,
            'neg': lambda x: -x,
            'abs': lambda x: sympy.Abs(x),
            'pow': lambda x, y: x**y,
            'sin': lambda x: sympy.sin(x),
            'cos': lambda x: sympy.cos(x),
            'inv': lambda x: 1 / x,
            'sqrt': lambda x: x**0.5,
            'self_exp': lambda x: sympy.exp(x**2),
            'square': lambda x: x**2
        }  # Converter set

sympy.sympify("mul(x, x)", locals=converter)
sympy.sympify('mul(x, x)', locals=converter)

mul(x, x)